# Header

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import os

exec(open("../header.py").read())

Header initialized


# Import

In [51]:
threshold = 40
threshold = str(threshold)

In [52]:
raw_train_data = pd.read_csv(processed_root("04-bow-added-features/feat_train_data-"+threshold+".csv"))
raw_val_data = pd.read_csv(processed_root("04-bow-added-features/feat_val_data-"+threshold+".csv"))
raw_test_data = pd.read_csv(processed_root("04-bow-added-features/feat_test_data-"+threshold+".csv"))

In [53]:
X_train = raw_train_data\
    .drop(['Unnamed: 0', 'poetry_text', 'poetry_author'], axis = 1)
X_val = raw_val_data\
    .drop(['Unnamed: 0', 'poetry_text', 'poetry_author'], axis = 1)
y_train = raw_train_data['poetry_author']
y_val = raw_val_data['poetry_author']

X_test = raw_test_data\
    .drop(['Unnamed: 0', 'poetry_text', 'poetry_author'], axis = 1)
y_test = raw_test_data['poetry_author']

In [54]:
len(X_train.columns)

1901

# Best model

In [94]:
model = RandomForestClassifier(max_depth = 24, class_weight = 'balanced', n_estimators = 500)

# Validation scores

In [95]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=24, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [96]:
# No single feature accounting for things
np.max(model.feature_importances_)

0.04890103577625357

In [97]:
# Most important word
X_train.iloc[0:1,np.argmax(model.feature_importances_):np.argmax(model.feature_importances_)+1]

,ave_line_len
0,4.528571


In [98]:
y_pred = model.predict(X_val)

In [99]:
accuracy = sum(y_pred == y_val)/len(y_val)

In [100]:
author_accuracy = pd.DataFrame({'pred':y_pred,
              'true':y_val})\
    .assign(correct = lambda x:x.pred == x.true)\
    .groupby('true', as_index = False)\
    .mean()\
    .loc[:,'correct']\
    .mean()

In [101]:
print("Accuracy: {:.3f}, Author-weighted Accuracy: {:.3f}".format(accuracy, author_accuracy))

Accuracy: 0.669, Author-weighted Accuracy: 0.659


# Test scores

In [117]:
X_train_combined = pd.concat([X_train, X_val], axis = 0, ignore_index = True)
y_train_combined = pd.concat([y_train, y_val], axis = 0, ignore_index = True)

In [118]:
model.fit(X_train_combined, y_train_combined)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=24, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [119]:
# No single feature accounting for things
np.max(model.feature_importances_)

0.049425886880792975

In [120]:
# Most important word
X_train.iloc[0:1,np.argmax(model.feature_importances_):np.argmax(model.feature_importances_)+1]

,ave_line_len
0,4.528571


In [121]:
y_pred = model.predict(X_test)

In [122]:
accuracy = sum(y_pred == y_test)/len(y_test)

In [123]:
author_accuracy = pd.DataFrame({'pred':y_pred,
              'true':y_test})\
    .assign(correct = lambda x:x.pred == x.true)\
    .groupby('true', as_index = False)\
    .mean()\
    .loc[:,'correct']\
    .mean()

In [124]:
print("Accuracy: {:.3f}, Author-weighted Accuracy: {:.3f}".format(accuracy, author_accuracy))

Accuracy: 0.661, Author-weighted Accuracy: 0.628
